# Ranking by Reordering

The problem of ranking a set of objects is ubiquitous not only in everyday life, but also for many scientific problems such as information retrieval, recommender systems, natural language processing, and drug discovery. In this turotial, we will rank footable teams based on the game scores.

In American collegiate football, it is traditional for each team to play one game against every other team in their division. We can use two-sided permutation Procrustes to use the results to rank the teams.
The data taken from *A. N. Langville, C. D. Meyer, Ranking by Reordering Methods, Princeton University Press, 2012, Ch. 8, pp. 97–112.* shows the pair-wise relationship for 5 football teams in the Coastal Division of the Atlantic Coast Conference. In Table 1, each team is given zero points for a game they lost (Duke lost to every other team in its division) and the point differential is used for games the team won. (E.g., Miami beats Duke by 45 points and UNC by 18 points.) 


|Table 1. Team by team game score differential|
|---------------------------------------|

| Team  | Duke | Miami | UNC | UVA | VT |
|-------|------|-------|-----|-----|----|
| Duke  | 0    | 0     | 0   | 0   | 0  |
| Miami | 45   | 0     | 18  | 8   | 20 |
| UNC   | 3    | 0     | 0   | 2   | 0  |
| UVA   | 31   | 0     | 0   | 0   | 0  |
| VT    | 45   | 0     | 27  | 38  | 0  |

These results are also summarized in the score-differential matrix $\mathbf{A}$ in **Fig. (ii)** based on the table given above,

![Fig. 1 Ranking by reordering with two-sided permutation with one-transformation](notebook_data/ranking_reordering/ranking.png "Fig. 1 Ranking by reordering with two-sided permutation with one-transformation")

Two-sided permutation Procrustes can be used to rank these teams, but one needs to define a proper target matrix. Traditionally, the rank-differential matrix has been used for this purpose and is defined for $n$ teams as, 

\begin{equation}
    \label{rank_differential}
    \mathbf{R}_{n \times n} =
    \begin{bmatrix}
        0 & 1 & 2 & \cdots & n-1 \\
        & 0 & 1 & \cdots & n-2 \\
        &   &\ddots &\ddots & \vdots \\
        &   &   & \ddots & 1 \\
        &   &   &        & 0
    \end{bmatrix}
\end{equation}

The rank-differential matrix $\mathbf{R} \in \mathbb{R}^{n \times n}$ is an upper-triangular matrix and its $ij$-th element specifies the difference in ranking between team $i$ and team $j$. This a sensible target for the score-differential matrix. Now, the two-sided permutation Procrustes method can be used to find the permutation matrix that maximizes the similarity between the score-differential matrix, $\mathbf{A}$, and the rank-differential matrix based on the definition of rank-differential matrix, $\mathbf{B}$ (**Fig. (ii)**)

\begin{equation}
    \min_{\mathbf{P}} {\left\lVert \mathbf{P}^{\top} \mathbf{A} \mathbf{P} - \mathbf{B}
        \right\rVert}_{F}^2
\end{equation}

This results to $[5,2,4,3,1]$ as the final rankings of the teams (**Fig. (iii)**).

In [ ]:
# only run this cell if you need to install the dependices
!pip install git+https://github.com/theochem/procrustes.git@master

In [1]:
import numpy as np

from procrustes import permutation_2sided

# input score-differential matrix
A = np.array([[ 0, 0, 0 ,  0,  0 ],    # Duke
              [45, 0, 18,  8,  20],    # Miami
              [ 3, 0, 0 ,  2,  0 ],    # UNC
              [31, 0, 0 ,  0,  0 ],    # UVA
              [45, 0, 27, 38,  0 ]])   # VT

# make rank-differential matrix
n = A.shape[0]
B = np.zeros((n, n))
for index in range(n):
    B[index, index:] = range(0, n - index)

# rank teams using two-sided Procrustes
result = permutation_2sided(A, B, single=True, 
                            mode='normal1', tol=10.e-6)

# compute teams' ranks
_, ranks = np.where(result.t == 1)
ranks += 1
print("Ranks = ", ranks)     # displays [5, 2, 4, 3, 1]

Ranks =  [5 2 4 3 1]


Why we need to add all the rank values by 1? Because Python's list index starts with 0, but we often index starting from 1 for physical objects.